# Monte Carlo Simulation for A/B Testing

## What is Monte Carlo Simulation?

Instead of running our A/B test once, we'll simulate it **10,000 times** to understand:
- How often would we get these results by random chance?
- What's the range of possible outcomes?
- How confident should we be in our decision?

**Think of it like this:** Instead of flipping a coin once, we flip it 10,000 times to truly understand the probability.

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Simulate A Single A/B Test

First, let's create a function that runs one A/B test.

In [ ]:
def run_single_ab_test(control_rate, treatment_rate, n_per_group):
    """
    Simulate a single A/B test.
    
    Returns:
    --------
    dict with conversion rates, lift, and p-value
    """
    # Generate conversions
    control_conversions = np.random.binomial(1, control_rate, n_per_group)
    treatment_conversions = np.random.binomial(1, treatment_rate, n_per_group)
    
    # Calculate rates
    control_observed = control_conversions.mean()
    treatment_observed = treatment_conversions.mean()
    lift = treatment_observed - control_observed
    
    # Statistical test
    t_stat, p_value = stats.ttest_ind(treatment_conversions, control_conversions)
    
    return {
        'control_rate': control_observed,
        'treatment_rate': treatment_observed,
        'lift': lift,
        'relative_lift': (treatment_observed / control_observed - 1) if control_observed > 0 else 0,
        'p_value': p_value,
        'significant': p_value < 0.05
    }

# Test it once
single_test = run_single_ab_test(
    control_rate=0.12,
    treatment_rate=0.145,
    n_per_group=5000
)

print("Single Test Result:")
print(f"Control Rate: {single_test['control_rate']:.2%}")
print(f"Treatment Rate: {single_test['treatment_rate']:.2%}")
print(f"Lift: {single_test['lift']:.2%}")
print(f"P-value: {single_test['p_value']:.4f}")
print(f"Significant: {single_test['significant']}")

## 3. Run 10,000 Simulations

Now let's run this test 10,000 times to see the **distribution of possible outcomes**.

In [ ]:
def run_monte_carlo_simulation(control_rate, treatment_rate, n_per_group, n_simulations=10000):
    """
    Run multiple A/B test simulations.
    
    This answers: "If we ran this test many times, what would we see?"
    """
    print(f"Running {n_simulations:,} simulations...")
    print(f"Each simulation: {n_per_group:,} users per group\n")
    
    results = []
    
    for i in range(n_simulations):
        if (i + 1) % 2000 == 0:
            print(f"  Progress: {i+1:,}/{n_simulations:,} ({(i+1)/n_simulations*100:.0f}%)")
        
        result = run_single_ab_test(control_rate, treatment_rate, n_per_group)
        results.append(result)
    
    print("\n✅ Simulation complete!\n")
    return pd.DataFrame(results)

# Run simulations
simulations = run_monte_carlo_simulation(
    control_rate=0.12,
    treatment_rate=0.145,
    n_per_group=5000,
    n_simulations=10000
)

## 4. Analyze Simulation Results

In [ ]:
print("="*70)
print(" " * 20 + "SIMULATION SUMMARY")
print("="*70)

# Basic statistics
print("\n📊 CONVERSION RATE DISTRIBUTIONS:")
print("-" * 70)
print("\nControl Group:")
print(f"  Mean: {simulations['control_rate'].mean():.2%}")
print(f"  Std Dev: {simulations['control_rate'].std():.2%}")
print(f"  Range: [{simulations['control_rate'].min():.2%}, {simulations['control_rate'].max():.2%}]")

print("\nTreatment Group:")
print(f"  Mean: {simulations['treatment_rate'].mean():.2%}")
print(f"  Std Dev: {simulations['treatment_rate'].std():.2%}")
print(f"  Range: [{simulations['treatment_rate'].min():.2%}, {simulations['treatment_rate'].max():.2%}]")

print("\n📈 LIFT ANALYSIS:")
print("-" * 70)
print(f"  Mean Lift: {simulations['lift'].mean():.2%}")
print(f"  Std Dev: {simulations['lift'].std():.2%}")
print(f"  Median Lift: {simulations['lift'].median():.2%}")
print(f"  5th Percentile: {simulations['lift'].quantile(0.05):.2%}")
print(f"  95th Percentile: {simulations['lift'].quantile(0.95):.2%}")

# Probability of positive lift
prob_positive = (simulations['lift'] > 0).mean()
print(f"\n  Probability of ANY positive lift: {prob_positive:.1%}")

# Probability of meaningful lift (>1%)
prob_meaningful = (simulations['lift'] > 0.01).mean()
print(f"  Probability of >1% lift: {prob_meaningful:.1%}")

# Statistical significance
prob_significant = simulations['significant'].mean()
print(f"\n🎯 STATISTICAL SIGNIFICANCE:")
print("-" * 70)
print(f"  Tests achieving p < 0.05: {prob_significant:.1%}")
print(f"  This is our 'statistical power': {prob_significant:.1%}")

if prob_significant >= 0.8:
    print(f"  ✅ Excellent! We have high power to detect this effect.")
elif prob_significant >= 0.6:
    print(f"  ⚠️  Moderate power. Consider increasing sample size.")
else:
    print(f"  ❌ Low power. We need more data to reliably detect this effect.")

print("\n" + "="*70)

## 5. Visualize the Distribution of Results

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Distribution of Control Rates
ax1 = fig.add_subplot(gs[0, 0])
ax1.hist(simulations['control_rate'] * 100, bins=50, color='#3498db', 
         alpha=0.7, edgecolor='black')
ax1.axvline(x=12, color='red', linestyle='--', linewidth=2, label='True Rate (12%)')
ax1.set_xlabel('Conversion Rate (%)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax1.set_title('Distribution of Control Group Rates', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Distribution of Treatment Rates
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(simulations['treatment_rate'] * 100, bins=50, color='#e74c3c', 
         alpha=0.7, edgecolor='black')
ax2.axvline(x=14.5, color='red', linestyle='--', linewidth=2, label='True Rate (14.5%)')
ax2.set_xlabel('Conversion Rate (%)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax2.set_title('Distribution of Treatment Group Rates', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Distribution of Lift
ax3 = fig.add_subplot(gs[1, :])
ax3.hist(simulations['lift'] * 100, bins=60, color='#2ecc71', 
         alpha=0.7, edgecolor='black')
ax3.axvline(x=2.5, color='red', linestyle='--', linewidth=2, label='True Lift (2.5%)')
ax3.axvline(x=0, color='black', linestyle='-', linewidth=2, alpha=0.5)
ax3.set_xlabel('Lift (%)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax3.set_title('Distribution of Treatment Lift (10,000 Simulations)', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Add percentile markers
p5 = simulations['lift'].quantile(0.05) * 100
p95 = simulations['lift'].quantile(0.95) * 100
ax3.axvline(x=p5, color='orange', linestyle=':', linewidth=2, alpha=0.7, label=f'5th-95th Percentile')
ax3.axvline(x=p95, color='orange', linestyle=':', linewidth=2, alpha=0.7)
ax3.legend()

# 4. P-value Distribution
ax4 = fig.add_subplot(gs[2, 0])
ax4.hist(simulations['p_value'], bins=50, color='#9b59b6', 
         alpha=0.7, edgecolor='black')
ax4.axvline(x=0.05, color='red', linestyle='--', linewidth=2, label='Significance Threshold (0.05)')
ax4.set_xlabel('P-value', fontsize=11, fontweight='bold')
ax4.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax4.set_title('Distribution of P-values', fontsize=13, fontweight='bold')
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

# 5. Significance Pie Chart
ax5 = fig.add_subplot(gs[2, 1])
sig_counts = simulations['significant'].value_counts()
colors = ['#2ecc71', '#e74c3c']
labels = [f'Significant\n({sig_counts[True]:,} tests)', 
          f'Not Significant\n({sig_counts[False]:,} tests)']
ax5.pie([sig_counts[True], sig_counts[False]], labels=labels, autopct='%1.1f%%',
        colors=colors, startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax5.set_title('Statistical Significance Rate', fontsize=13, fontweight='bold')

plt.suptitle('Monte Carlo Simulation Results: 10,000 A/B Tests', 
             fontsize=16, fontweight='bold', y=0.995)
plt.show()

## 6. Confidence Intervals from Simulation

Monte Carlo gives us another way to calculate confidence intervals - just take percentiles!

In [ ]:
print("="*70)
print(" " * 15 + "SIMULATION-BASED CONFIDENCE INTERVALS")
print("="*70)

# 95% CI for lift
ci_lower = simulations['lift'].quantile(0.025)
ci_upper = simulations['lift'].quantile(0.975)

print("\n95% Confidence Interval for Lift:")
print(f"  [{ci_lower:.2%}, {ci_upper:.2%}]")

# 90% CI for lift
ci_90_lower = simulations['lift'].quantile(0.05)
ci_90_upper = simulations['lift'].quantile(0.95)

print("\n90% Confidence Interval for Lift:")
print(f"  [{ci_90_lower:.2%}, {ci_90_upper:.2%}]")

# Interpretation
print("\n💡 What This Means:")
print("-" * 70)
if ci_lower > 0:
    print("  ✅ We're 95% confident the treatment INCREASES conversions")
    print(f"     The true lift is likely between {ci_lower:.2%} and {ci_upper:.2%}")
elif ci_upper < 0:
    print("  ❌ We're 95% confident the treatment DECREASES conversions")
    print(f"     The true lift is likely between {ci_lower:.2%} and {ci_upper:.2%}")
else:
    print("  ⚠️  The confidence interval includes zero")
    print("     We can't be confident there's any real effect")

print("\n" + "="*70)

## 7. Risk Analysis

Monte Carlo helps us quantify **risk**: What's the probability of making a wrong decision?

In [ ]:
print("="*70)
print(" " * 25 + "RISK ANALYSIS")
print("="*70)

# Risk scenarios
print("\n🎲 PROBABILITY OF DIFFERENT OUTCOMES:")
print("-" * 70)

scenarios = [
    ("Lift > 3%", simulations['lift'] > 0.03),
    ("Lift > 2%", simulations['lift'] > 0.02),
    ("Lift > 1%", simulations['lift'] > 0.01),
    ("Any positive lift", simulations['lift'] > 0),
    ("Negative lift", simulations['lift'] < 0),
    ("Lift < -1%", simulations['lift'] < -0.01),
]

for scenario_name, condition in scenarios:
    probability = condition.mean()
    print(f"  {scenario_name:.<30} {probability:>6.1%}")

# Decision risk
print("\n\n⚠️  DECISION RISKS:")
print("-" * 70)

# Type I Error (False Positive)
# If there's truly NO effect, how often would we incorrectly say there is?
print("\nType I Error (False Positive Risk):")
print("  If we run this test with NO real effect, we'd still get")
print("  a 'significant' result ~5% of the time (our alpha level)")

# Type II Error (False Negative)
false_negative_rate = (simulations['significant'] == False).mean()
print("\nType II Error (False Negative Risk):")
print(f"  Even with a REAL effect, we fail to detect it {false_negative_rate:.1%} of the time")
print(f"  This means our statistical power is {1-false_negative_rate:.1%}")

# Business risk
prob_loss = (simulations['lift'] < 0).mean()
avg_loss_when_negative = simulations[simulations['lift'] < 0]['lift'].mean()

print("\n💰 BUSINESS RISK:")
print("-" * 70)
print(f"  Probability treatment HURTS conversions: {prob_loss:.1%}")
if prob_loss > 0:
    print(f"  Average loss when negative: {avg_loss_when_negative:.2%}")
    
    # Expected value calculation
    expected_lift = simulations['lift'].mean()
    print(f"\n  Expected lift (accounting for risk): {expected_lift:.2%}")

print("\n" + "="*70)

## 8. Sample Size Impact

Let's see how sample size affects our confidence.

In [ ]:
print("Comparing different sample sizes...\n")

sample_sizes = [1000, 2500, 5000, 10000]
power_results = []

for n in sample_sizes:
    print(f"Testing with {n:,} users per group...")
    
    # Run smaller simulation (1000 tests is enough to estimate power)
    temp_sims = run_monte_carlo_simulation(
        control_rate=0.12,
        treatment_rate=0.145,
        n_per_group=n,
        n_simulations=1000
    )
    
    power = temp_sims['significant'].mean()
    ci_width = temp_sims['lift'].quantile(0.975) - temp_sims['lift'].quantile(0.025)
    
    power_results.append({
        'sample_size': n,
        'power': power,
        'ci_width': ci_width
    })
    
    print(f"  Power: {power:.1%}")
    print(f"  95% CI Width: {ci_width:.2%}\n")

power_df = pd.DataFrame(power_results)

In [ ]:
# Visualize sample size impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Power vs sample size
ax1 = axes[0]
ax1.plot(power_df['sample_size'], power_df['power'] * 100, 
         marker='o', linewidth=3, markersize=10, color='#2ecc71')
ax1.axhline(y=80, color='red', linestyle='--', linewidth=2, label='80% Power Target')
ax1.set_xlabel('Sample Size per Group', fontsize=12, fontweight='bold')
ax1.set_ylabel('Statistical Power (%)', fontsize=12, fontweight='bold')
ax1.set_title('How Sample Size Affects Power', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# CI width vs sample size
ax2 = axes[1]
ax2.plot(power_df['sample_size'], power_df['ci_width'] * 100, 
         marker='o', linewidth=3, markersize=10, color='#3498db')
ax2.set_xlabel('Sample Size per Group', fontsize=12, fontweight='bold')
ax2.set_ylabel('95% CI Width (%)', fontsize=12, fontweight='bold')
ax2.set_title('How Sample Size Affects Precision', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("  As sample size increases:")
print("    ✓ Statistical power increases (more likely to detect real effects)")
print("    ✓ Confidence intervals get narrower (more precise estimates)")

## 9. Final Recommendation with Monte Carlo Insights

In [ ]:
print("="*70)
print(" " * 15 + "MONTE CARLO-INFORMED DECISION")
print("="*70)

print("\n📊 SIMULATION RESULTS SUMMARY:")
print("-" * 70)
print(f"  Based on 10,000 simulated A/B tests:")
print(f"  • Expected lift: {simulations['lift'].mean():.2%}")
print(f"  • 95% CI: [{simulations['lift'].quantile(0.025):.2%}, {simulations['lift'].quantile(0.975):.2%}]")
print(f"  • Statistical power: {simulations['significant'].mean():.1%}")
print(f"  • Probability of positive effect: {(simulations['lift'] > 0).mean():.1%}")
print(f"  • Probability of >1% lift: {(simulations['lift'] > 0.01).mean():.1%}")

# Make recommendation
prob_positive = (simulations['lift'] > 0).mean()
power = simulations['significant'].mean()
ci_lower = simulations['lift'].quantile(0.025)

print("\n\n🎯 FINAL RECOMMENDATION:")
print("="*70)

if prob_positive >= 0.95 and power >= 0.8 and ci_lower > 0:
    print("\n✅ STRONG RECOMMENDATION: PROCEED WITH ROLLOUT")
    print("\nReasoning:")
    print(f"  1. Very high probability ({prob_positive:.1%}) of positive effect")
    print(f"  2. Adequate statistical power ({power:.1%})")
    print(f"  3. 95% confident effect is positive (CI doesn't include zero)")
    print("\nAction: Roll out the new button to all users.")
    
elif prob_positive >= 0.85 and power >= 0.7:
    print("\n✅ MODERATE RECOMMENDATION: PROCEED WITH CAUTION")
    print("\nReasoning:")
    print(f"  1. Good probability ({prob_positive:.1%}) of positive effect")
    print(f"  2. Acceptable power ({power:.1%})")
    print("\nAction: Gradual rollout with monitoring, or extend test for more data.")
    
else:
    print("\n⚠️  RECOMMENDATION: DO NOT PROCEED YET")
    print("\nReasoning:")
    print(f"  1. Probability of positive effect: {prob_positive:.1%}")
    print(f"  2. Statistical power: {power:.1%}")
    print("\nAction: Extend the test to collect more data or try a different treatment.")

print("\n" + "="*70)

## 10. Key Takeaways

### What Monte Carlo Simulation Teaches Us:

1. **Quantifies Uncertainty:** We see the full range of possible outcomes, not just one test result
2. **Validates Sample Size:** Shows whether we have enough data to make confident decisions
3. **Estimates Risk:** Calculates probability of different scenarios (positive, negative, meaningful lift)
4. **Alternative to Math:** Provides confidence intervals through simulation rather than formulas
5. **Builds Intuition:** Seeing 10,000 tests helps understand statistical concepts

### Real-World Applications:

- **Finance:** Risk modeling for investment portfolios
- **Engineering:** Reliability testing and quality control
- **Healthcare:** Drug trial planning and analysis
- **Business:** Revenue forecasting with uncertainty

### Interview Talking Points:

> "Rather than relying on a single test, I ran 10,000 Monte Carlo simulations to understand the full distribution of possible outcomes. This revealed we have 95%+ probability of a positive effect and adequate statistical power, giving us high confidence in the recommendation."

---

*Monte Carlo simulation demonstrates advanced statistical thinking and computational skills - perfect for decision science roles at companies like Disney.*